LangChain을 활용한 모델 사용, 비용 모니터링 및 캐싱 전략
    GPT-4o-mini GPT-3.5-turbo 비용이 60% 저렴
    Langchain V0.3x 부터 openAI가 별도 패키지로 분리되어 있어서 필요한 패키치 설치해야함: langchain-openai 필요
    토큰사용량 추적, 캐싱을 위한 langchain-community도 별도 설치
    환경변수 변수 관리 패키지 : python-dotenv

In [1]:
%pip install langchain-openai langchain-community python-dotenv openai

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 16.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.6 MB 9.5 MB/s eta 0:00:02
   --------- ------------------------------ 3.1/12.6 MB 7.7 MB/s eta 0:00:02
   ----------------- ---------------------- 5.5/12.6 MB 8.5 MB/s eta 0:00:01
   --------------------------- ------------ 8.7/12.6 MB 10.1 MB/s eta 0:00:01
   --------------------------------- ------ 10.5/12.6 MB 9.7 MB/s eta 0:00:01
   ---------------------------------------  12.3/12.6 MB 9.7 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 9.5 MB/s eta 0:00:00

   -- -------------------------------------  1/17 [propcache]
   ---- -----------------------------------  2/17 [numpy]
   ---- -----------------------------------  2/17 [numpy]
   ---- -----------------------------------  2/17 [numpy]
   -

In [2]:
 # 환경변수 로드
from dotenv import load_dotenv
load_dotenv() 

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.7)
prompt = "Langchain에 대해 한 문장으로 설명해줘"
result = llm.invoke(prompt)
result.content 

'LangChain은 언어 모델을 활용해 다양한 애플리케이션을 구축할 수 있도록 돕는 프레임워크로, 데이터 소스와의 통합, 사용자 인터페이스 설계 등을 지원합니다.'

In [ ]:
#사용량
result.usage_metadata 

{'input_tokens': 18,
 'output_tokens': 47,
 'total_tokens': 65,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [5]:
# 콜백 함수를 통한 누적 토큰 추적(get_openai_callback)
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫번째 호출
    res1 = llm.invoke('서울의 오늘 날씨는 어떤지 알려줘?')
    print('응답1', res1.content[:10], '...')
    # 두번째 호출
    res2 = llm.invoke('파이썬으로 랭체인 사용법을 알려줘')
    print('응답2', res2.content[:10], '...')

# 누적 토큰 사용량 출력 콜백 cb에는 블록 내 전체 토큰 사용량이 누적
# 총 토큰 수 
print('총 토큰 수 :', cb.total_tokens)
# 프롬프트 토큰 수 
print('프롬프트 토큰 수 :', cb.prompt_tokens)
# 응답 토큰 수 
print('응답 토큰 수 :', cb.completion_tokens)
# 비용 계산
print('비용(USD) :', cb.total_cost)


응답1 죄송하지만, 실시간 ...
응답2 랭체인(LangCh ...
총 토큰 수 : 740
프롬프트 토큰 수 : 39
응답 토큰 수 : 701
비용(USD) : 0.00042645


In [ ]:
#LangChain의 LLM 응답캐싱 (InMemory Cache, SQLiteCache)
# 사용하는 이유: 동일한 질문은 저장해뒀다가 응답에 사용
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCash 설정
set_llm_cache(InMemoryCache()) 

In [14]:
# 캐시 사용 전후를 비교, 같은 질문을 두번 호출
query = '재미있는 유머 하나 알려줘'
# 첫 번째 호출 (캐시에 없으면 api호출 발생)
result1 = llm.invoke(query)
print(f'응답 1 : {result1.content}')
print('****************************************************')
# 첫 번째 호출 (캐시에 없으면 api호출 발생)
result2 = llm.invoke(query)
print(f'응답 1 : {result2.content}')


응답 1 : 물론이죠! 여기 간단한 유머 하나 있어요:

왜 컴퓨터는 춤을 잘 못 출까요?

바이러스에 걸릴까 봐 두려워서요! 😄

재미있으셨나요? 더 필요하시면 말씀해 주세요!
****************************************************
응답 1 : 물론이죠! 여기 간단한 유머 하나 있어요:

왜 컴퓨터는 춤을 잘 못 출까요?

바이러스에 걸릴까 봐 두려워서요! 😄

재미있으셨나요? 더 필요하시면 말씀해 주세요!


In [15]:
# 실행시간 측정
import time
# 첫 번재 호출 시간
query = '점심메뉴 추천해줘'
start = time.time(); llm.invoke(query); end = time.time()
print(f'첫번째 호출시간: {end-start}')

start = time.time(); llm.invoke(query); end = time.time()
print(f'두번째 호출시간: {end-start}') #거의 0초 예상, invoke했지만 api안가고 그냥 cache 메모리 안에 있는걸 꺼낸거니까
# 이게 InMemory 방식. 시간비용 절약할 수 있고, 중복대답을 방지할 수 있다

첫번째 호출시간: 7.167054176330566
두번째 호출시간: 0.0008411407470703125


In [ ]:
# SQLite 캐시(디스크기반 캐시)
import os
from langchain_community.cache import SQLiteCache
#기존 캐시 DB 삭제(.langchain.db 초기화)



#아래 코드는 처음실행만 돌리고 두번째 실행부터는 주석처리하기기 
#if os.path.exists('.langchain.db'):
#  os.remove('.langchain.db')



# SQLite 캐시 설정(지정한 경로의 DB 파일을 생성 / 사용)
set_llm_cache(SQLiteCache(database_path='.langchain.db'))  # langchain.db있으면 사용 없으면 생성

# 동일한 query를 두번 호출해서 결과와 시간을 비교
query = '썰렁한 유머 하나 알려줘'
# 첫번째 호출(캐시에 없으면 api 호출 발생)
start = time.time(); result1 = llm.invoke(query); end = time.time()
print(f'첫 번째 호출시간 : {end-start}')
print(f'응답1 : {result1.content}')

# 두번째 호출(동일한 query, 캐시를 확인하고 종일 질문이면 api 미호출)
start = time.time(); result2 = llm.invoke(query); end = time.time()
print(f'두두 번째 호출시간 : {end-start}')
print(f'응답2 : {result2.content}')  # 일관성있는 답변을 해주는 장점이 있음.

첫 번째 호출시간 : 2.278498649597168
응답1 : 왜 바나나는 길을 건너지 않았을까요?

바나나 껍질이 미끄러워서요! 😄
두두 번째 호출시간 : 0.0012407302856445312
응답2 : 왜 바나나는 길을 건너지 않았을까요?

바나나 껍질이 미끄러워서요! 😄
